In [12]:
from __future__ import print_function
import os, sys
import pickle
import os.path
import io
import json

import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as scs

from matplotlib import pyplot as plt
import seaborn as sns


from io import StringIO
from math import floor
from tqdm import tqdm, trange

from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec


from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2 import service_account


class google_api():
    
    def __init__(self, token_file = 'token.pickle',fauth = None):
        
        self.SCOPES = (
            #with ability to use files/images stored in drive
            'https://www.googleapis.com/auth/drive',
            #to edit slides
            'https://www.googleapis.com/auth/presentations',
            #to sheet
            'https://www.googleapis.com/auth/spreadsheets.readonly'
        )
        
        #get credential file
        self.HOME = os.environ.get('HOME')
        
        if fauth is None:
            fauth = 'bioreport.json'

        if os.path.exists('./.env'):
            path = './.env'
        else:
            path = os.path.join(self.HOME, '.credentials')
            
        self.fauth = os.path.join(path, fauth)            

        self.token_file = token_file

        auth = json.load(open(self.fauth,'r'))
        if auth.get("type",'')=="service_account":
            #print(f'****** using service account: {self.fauth}')
            self.creds = self.get_service_account()
        else:
            self.creds = self.get_token()

    def get_service_account(self):
        return service_account.Credentials.from_service_account_file(self.fauth, scopes=self.SCOPES)
        
    
    def get_service(self,name=['drive','sheet','slide']):
        
        s = {}            
        if 'sheet' in name: 
            s['sheet'] = build('sheets', 'v4', credentials=self.creds)
            
        if 'drive' in name:        
            s['drive'] =  build('drive',  'v3', credentials=self.creds)
            
        if 'slide' in name:
            s['gslide'] = build('slides', 'v1', credentials=self.creds)
        
        return s

    
class gsheet(google_api):
    
    def __init__(self,sheetid, ftoken = 'token.pickle',
                 fauth = None,
                 sheetname = None):        
        #        
        super().__init__(fauth = fauth, token_file = ftoken)
        #print(ftoken, type(ftoken))
        #
        self.sheetid = sheetid
        self.sheetname = sheetname
        
        s = self.get_service()
        self.drive = s['drive'] 
        self.gsheet = s['sheet']    
        
        self.sheet = self.gsheet.spreadsheets()
 

    def get_sheet_df(self,name, start='A',
                     end=None, iheader=0, idata=0, lastrow=0,    
                     major='COLUMNS'):  
        
        #get header
        if iheader>0:
            hrange = f'{name}!{start}{iheader}:{end}{iheader}'
            result = self.sheet.values().get(spreadsheetId=self.sheetid, 
                                                    range=hrange).execute()
            if idata==1:
                idata=2            
            self.all_columns = result.get('values', [])[0]
        else:
            iheader=0
            nrows = lastrow-idata+1
            self.all_columns=['']*nrows
            
        
        
        #get data
        if lastrow==0:
            if end is None:
                drange = f'{name}'
            else:
                drange = f'{name}!{start}:{end}'
        else:
            drange = f'{name}!{start}{idata}:{end}{lastrow}'

        result = self.sheet.values().get(spreadsheetId=self.sheetid, 
                                             range=drange).execute()
         
        values = result['values']

        if lastrow==0:
            df = pd.DataFrame(values).transpose()
            col = df.iloc[0].values
            df = df.drop([0])
            df.columns = col
        else:
            nrows = lastrow-idata+1
            data = {col:['']*nrows for col in self.key_columns}
            for col in self.key_columns:
                if col in self.all_columns:
                    ic = self.all_columns.index(col)
                    v = values[ic]
                    nv = len(v)
                    data[col][0:nv] = v
                        
            df = pd.DataFrame.from_dict(data)
        

        #df = df[:,self.key_columns].dropna(axis='columns', thresh=0.95) 
        
        return df

    def sheet_update(self,body, major='COLUMNS'):  
        
        result = self.sheet.values().batchUpdate(spreadsheetId=self.sheetid, 
                                                 body=body).execute()

        print('{0} cells updated.'.format(result.get('totalUpdatedCells')))        
        
        return result

    def single_update(self,name, loc='A1', major='COLUMNS'):  
        

        
        #get data
        #Ref: https://developers.google.com/sheets/api/guides/values
        value_input_option = 'RAW'
        #drange = f'{name}!{start}{istart}:{end}{iend}'
        drange = f'{name}!{loc}'
        result = self.sheet.values().update(spreadsheetId=self.sheetid, 
                                            range=drange,
                                            valueInputOption=value_input_option,
                                            body=body,
                                            majorDimension=major).execute()

        print('{0} cells updated.'.format(result.get('updatedCells')))        
        
        return result                   

In [14]:
gs = gsheet(sheetid = '1nENiuyUfHexN584sMmBJ8y1d5cuyijrZ2fkd9XFSdVE', 
            fauth = 'coronaresponse_10ac.json')
#    
df = gs.get_sheet_df('AfricaInfluencers')
df

,Name,Jeffrey Gettleman,Africa24 Media,Scapegoat,Africa Check,James Copnall,Online Africa,Patrick Ngowi,DOS African Affairs,MoadowAJE,...,Computicket,loyiso gola,5FM,mailandguardian,Helen Zille,Julius Sello Malema,News24,Jacob G,Gareth Cliff,Trevor Noah
1,twitter_name,@gettleman,@a24media,@andiMakinana,@AfricaCheck,@JamesCopnall,@oafrica,@PatrickNgowi,@StateAfrica,@Moadow,...,@Computicket,@loyisogola,@5FM,@mailandguardian,@helenzille,@Julius_S_Malema,@News24,@SAPresident,@GarethCliff,@Trevornoah
2,Country,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [15]:
df.T

,1,2
Name,twitter_name,Country
Jeffrey Gettleman,@gettleman,None
Africa24 Media,@a24media,None
Scapegoat,@andiMakinana,None
Africa Check,@AfricaCheck,None
James Copnall,@JamesCopnall,None
Online Africa,@oafrica,None
Patrick Ngowi,@PatrickNgowi,None
DOS African Affairs,@StateAfrica,None
MoadowAJE,@Moadow,None
